## 2.3 训练技巧与实战
前面我们对神经网络的基本训练过程都进行了详细的介绍，但实践过程当中，我们通常会遇到很多情况导致我们的神经网络训练到某个程度之后就无法继续优化（前进），如何解决这个问题是2.3所要探讨的重点，我们主要分为：
* 局部最小值(local minima)和鞍点(saddle points)
* 批次(batch)与动量(momentum)
* 自动调整学习率(learning rate)  (*关于学习率的一些优化算法，会在后面的章节进行讲解与实践*)
* 损失函数选择(loss function)  (*Part1已做过介绍这里不再讲解*)
* 批量标准化(batch normalization)

*实战部分*为**两个比赛**：
> 训练过程建议读者在[Colab](https://colab.research.google.com/)或者[Kaggle](https://www.kaggle.com/)上选用GPU加速跑模型。
* [COVID-19 Cases Prediction](https://www.kaggle.com/c/ml2021spring-hw1)
* [TIMIT framewise phoneme classification](https://www.kaggle.com/c/ml2021spring-hw2)

In [52]:
import torch
from torch import nn
from d2l import torch as d2l
import math

### 2.3.1 训练技巧

在讲解训练中遇到的问题之前，我们现在对训练的*架构(Framework)*进行一下总览：

**Training**
Training data: $\left\{\left(x^{1}, \hat{y}^{1}\right),\left(x^{2}, \hat{y}^{2}\right), \ldots,\left(x^{N}, \hat{y}^{N}\right)\right\}$

训练步骤(Training Steps):
* Step 1: 初始化模型参数，$y=f_\theta(x)$
* Step 2: 定义损失函数,$L(\theta)$
* Step 3: 优化，$\boldsymbol{\theta}^{*}=\arg \min _{\boldsymbol{\theta}} L$

**Testing**
Testing data: $\left\{x^{N+1}, x^{N+2}, \ldots, x^{N+M}\right\}$
* 使用$f_{\theta^*}(x)$预测测试集的标签。

![](https://s2.loli.net/2022/01/18/cX2tj8ULx6M3SwK.png)


**过拟合（Overfitting）**
如下图所示，训练集误差减小，但测试集误差很大，往往发生了过拟合的现象：
![](https://s2.loli.net/2022/01/18/5sMrJuzXgRk3xH9.png)

* 在数据层面上，解决方法就是训练*更多的数据*：
![](https://s2.loli.net/2022/01/18/FoBqPgenW7NkIRa.png)
* 而在模型层面上的解决方法有:
    * 更少的参数，或者共享参数(简化模型)
    ![](https://s2.loli.net/2022/01/18/JKjtgfakyQ2C54o.png)
    * 更少的特征
    * Early Stopping
    * 正则化(*Regularization*)
    * Dropout
    * 一个经典的例子就是CNN(*卷积神经网络*)
![](https://s2.loli.net/2022/01/18/v2f6GHNK49lXw5Y.png)

#### 权重初始化
* **期望为0**
* 输入与输出**方差一样**，所以$n_{t-1} \gamma_{t}=1$。 因为 $h^t$ 是由 $t-1$ 层的 $n$ 个参数 $w$ 运算求得的，而 $t-1$ 层的这些参数之前假设了他们都是服从方差为 $\gamma$ 的分布，所以他们相加就成了 $n_{t-1}\gamma$, 了。
$$
\begin{aligned}
\operatorname{Var}\left[h_{i}^{t}\right] &=\mathbb{E}\left[\left(h_{i}^{t}\right)^{2}\right]-\mathbb{E}\left[h_{i}^{t}\right]^{2}=\mathbb{E}\left[\left(\sum_{j} w_{i, j}^{t} h_{j}^{t-1}\right)^{2}\right] \\
&=\mathbb{E}\left[\sum_{j}\left(w_{i, j}^{l}\right)^{2}\left(h_{j}^{t-1}\right)^{2}+\sum_{j \neq k} w_{i, j}^{l} w_{i, k}^{t} h_{j}^{t-1} h_{k}^{t-1}\right] \\
&=\sum_{j} \mathbb{E}\left[\left(w_{i, j}^{l}\right)^{2}\right] \mathbb{E}\left[\left(h_{j}^{t-1}\right)^{2}\right] \\
&=\sum_{j} \operatorname{Var}\left[w_{i, j}^{t}\right] \operatorname{Var}\left[h_{j}^{t-1}\right]=n_{t-1} \gamma_{t} \operatorname{Var}\left[h_{j}^{t-1}\right]
\end{aligned}
$$

综上所述，可以推导出：
$$
\begin{aligned}
&\frac{\partial \ell}{\partial \mathbf{h}^{t-1}}=\frac{\partial \ell}{\partial \mathbf{h}^{t}} \mathbf{W}^{t} =>\quad\left(\frac{\partial \ell}{\partial \mathbf{h}^{t-1}}\right)^{T}=\left(W^{t}\right)^{T}\left(\frac{\partial \ell}{\partial \mathbf{h}^{t}}\right)^{T} \\
&\mathbb{E}\left[\frac{\partial \ell}{\partial h_{i}^{t-1}}\right]=0 \\
&\operatorname{Var}\left[\frac{\partial \ell}{\partial h_{i}^{t-1}}\right]=n_{t} \gamma_{t} \operatorname{Var}\left[\frac{\partial \ell}{\partial h_{j}^{t}}\right] \quad=>n_{t} \gamma_{t}=1
\end{aligned}
$$


**$Xavier$初始化**
实际上，输入输出是很难控制的，难以满足$n_{t-1} \gamma = 1 和 n_t \gamma=1$
Xavier使得
$$\gamma_{t}\left(n_{t-1}+n_{t}\right) / 2=1 \quad \rightarrow \gamma_{t}=2 /\left(n_{t-1}+n_{t}\right)$$
为了适配权重变化，特别是$n_t$。*正态分布*与*均匀分布*表示如下所示：
$$\begin{aligned}
&\mathcal{N}\left(0, \sqrt{2 /\left(n_{t-1}+n_{t}\right)}\right) \\
&\mathscr{U}\left(-\sqrt{6 /\left(n_{t-1}+n_{t}\right)}, \sqrt{6 /\left(n_{t-1}+n_{t}\right)}\right) \text { 分布 } \mathscr{U}[-a, a] \text { 和方差是 } a^{2} / 3
\end{aligned}$$

代码实现如下所示:

In [5]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=math.sqrt(2/(m.in_features+m.out_features)))
nn.Sequential(nn.Linear(2, 1)).apply(init_weights)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

#### 更小的梯度

##### 局部最小值（Local minimal）与鞍点（Saddle point）
损失函数在*局部最小值*和*鞍点*的时候，梯度大小都会为0，但两者显著的区别如下图所示:
![](https://s2.loli.net/2022/01/18/JqjvaE7N9w841WP.png)

我们可以清楚的看到，鞍点的位置我们是有路可走的，但在局部最小值的地方我们会陷入一个“峡谷”当中。换而言之，鞍点情况下进行优化比在局部最小值继续优化更为*简单*。

为此我们需要借助数学的工具对这两种情况进行判定, 可见[推理过程](http://www.offconvex.org/2016/03/22/saddlepoints/)

实际情况下，通过大量的实验证明，我们的模型会更多的处在鞍点的位置，而并非局部最小值处，因此训练过程中，我们完全可以大胆的进行梯度的调节。

![](https://s2.loli.net/2022/01/18/4aZnJARtYj3UsB7.png)

##### 批次（Batch）

在$\boldsymbol{\theta}^{*}=\arg \min _{\boldsymbol{\theta}} L$过程当中，使用批次训练过程如下：
![](https://s2.loli.net/2022/01/18/gnLA5QKtkxHyerl.png)

实际上考虑到并行计算的因素，**大的批次**对训练时间是*没有显著的影响*（除非特别大的Batch Size），但**小的批次**运行完一个epoch需要花费*更长的时间*。

![](https://s2.loli.net/2022/01/18/TWlGrud1R4MUYAJ.png)

在*MNIST*和*CIFAR-10*的两个数据集当中，批次大小与准确度的关系如下所示：
![](https://s2.loli.net/2022/01/18/qX24w7KSTyBGQ1n.png)

所以Batch Size的合理设置十分重要，下面是关于一些Batch Size大小的对比：

| Batch Size           | Small      | Large                |
| -------------------- | ---------- | -------------------- |
| Speed for one update | Same       | Same (not too large) |
| Time for one epoch   | Slower     | **Faster**           |
| Gradient             | Noisy      | Stable               |
| Optimization         | **Better** | Worse                |
| Generalization       | **Better** | Worse                |


##### 动量(Momentum)
$m^t=\lambda m^{t-1} - \eta g^{t-1}$, $m^0=0$
使用动量前后对比：
* 前:
![](https://s2.loli.net/2022/01/18/aTc49M7XvFzASOo.png)
* 后
![](https://s2.loli.net/2022/01/18/eZUbiRzYgIxCo9s.png)


在实际例子当中，动量可以让我们更容易跳出局部最小值，使得模型可以继续优化下去:
![](https://s2.loli.net/2022/01/18/vx4gGSVd9uYIrnC.png)

#### 批量标准化(*BN*)
> 几乎所有卷积神经网络都会使用**批量归一化**, 当然另外还有[layer norm](https://arxiv.org/abs/1607.06450)

##### 原理
仅仅对原始输入数据进行标准化是不充分的，因为虽然这种做法可以保证原始输入数据的质量，但它却无法保证隐藏层输入数据的质量。浅层参数的微弱变化经过多层线性变换与激活函数后被放大，改变了每一层的输入分布，造成深层的网络需要不断调整以适应这些分布变化，最终导致模型**难以训练收敛**。

简单的将每层得到的数据进行直接的标准化操作显然是不可行的，因为这样会*破坏每层自身学到的数据特征*。为了使“规范化”之后不破坏层结构本身学到的特征，BN引入了**两个**可以学习的“重构参数”以期望能够从规范化的数据中重构出层本身学到的特征。

下面$\gamma$为方差，$\beta$为均值

* 计算批处理数据均值
$$\mu_{B}=\frac{1}{|B|} \sum_{i \in B} x_{i}$$
* 计算批处理数据方差
$$\sigma_{B}^{2}=\frac{1}{|B|} \sum_{i \in B}\left(x_{i}-\mu_{B}\right)^{2}+\epsilon$$
* 规范化
$$x_{i+1}=\gamma \frac{x_{i}-\mu_{B}}{\sigma_{B}}+\beta$$


##### 使用
作用在：
* 全连接层和卷积层输出上，**激活函数前**
* 全连接层和卷积层输入上
* 对于全连接层，作用在*特征*维
* 对于卷积层，作用在*通道*维

使用BN后，可以:
* 缓解梯度消失，加速网络收敛。
* 简化调参，网络更稳定。BN层抑制了参数微小变化随网络加深而被放大的问题，对参数变化的适应能力更强，更容易调参。
* 防止过拟合。BN层将每一个batch的均值和方差引入到网络中，由于每个batch的这俩个值都不相同，可看做为训练过程增 加了随机噪声，可以起到一定的正则效果，防止过拟合。
* 另外后续有论文指出它可能就是通过加入噪音来控制模型复杂度，因此**没必要**与`Dropout`混合使用

下面我们通过代码实现：
* 手动从零开始实现

In [54]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    """
    :param X:
    :param gamma:
    :param beta:
    :param moving_mean: 全局均值
    :param moving_var: 权值方差
    :param eps:
    :param momentum:
    :return:
    """
    if not torch.is_grad_enabled():
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)  # 2 为全连接， 4 为卷积
        if len(X.shape) == 2:
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 2d卷积
            mean = X.mean(dim=(0, 2, 3), keepdim=True)  # (0：批量大小，1输入输出通道， 2高， 3宽)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)  # 4D
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 全局更新
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta
    return Y, moving_mean.data, moving_var.data

撇开算法细节，注意我们实现层的基础设计模式。 通常情况下，我们用一个单独的函数定义其数学原理，比如说batch_norm。 然后，我们将此功能集成到一个自定义层中，其代码主要处理：
1. 数据移动到训练设备（如GPU）
2. 分配和初始化任何必需的变量
3. 跟踪移动平均线（此处为均值和方差）等问题。
为了方便起见，我们并不担心在这里自动推断输入形状，因此我们需要指定整个特征的数量。

创建一个BatchNorm层:

In [55]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features,  1, 1)
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        self.moving_mean = torch.zeros(shape)  # 这两个参数不用迭代，就放入parameter了
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        Y, self.moving_mean, self.moving_var = batch_norm(X, self.gamma,
                                                          self.beta,
                                                          self.moving_mean,
                                                          self.moving_var,
                                                          eps=1e-5,   # 常见设定
                                                          momentum=0.9)
        return Y

为了简单起见，我们将BN层作用在LeNet上:

In [82]:
net1 = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), BatchNorm(6, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), BatchNorm(16, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(16*4*4, 120), BatchNorm(120, num_dims=2), nn.Sigmoid(),
    nn.Linear(120, 84), BatchNorm(84, num_dims=2), nn.Sigmoid(),
    nn.Linear(84, 10))

* Pytorch API实现
1. 对2d或3d数据进行批标准化（Batch Normlization）操作：
`class torch.nn.BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True)`
`num_features`：特征的维度 (N,L) -> L ;(N,C,L) -> C
2.对由3d数据组成的4d数据（N,C,X,Y）进行Batch Normlization：
`class torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True)`
`num_features`：特征的维度 (N,C,X,Y) -> C

In [93]:
net2 = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), nn.BatchNorm2d(6), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.BatchNorm2d(16), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(256, 120), nn.BatchNorm1d(120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.BatchNorm1d(84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [96]:
X = torch.randn(1, 1, 28, 28)
net2.eval()
for layer in net2:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2d output shape:	 torch.Size([1, 6, 24, 24])
BatchNorm2d output shape:	 torch.Size([1, 6, 24, 24])
Sigmoid output shape:	 torch.Size([1, 6, 24, 24])
AvgPool2d output shape:	 torch.Size([1, 6, 12, 12])
Conv2d output shape:	 torch.Size([1, 16, 8, 8])
BatchNorm2d output shape:	 torch.Size([1, 16, 8, 8])
Sigmoid output shape:	 torch.Size([1, 16, 8, 8])
AvgPool2d output shape:	 torch.Size([1, 16, 4, 4])
Flatten output shape:	 torch.Size([1, 256])
Linear output shape:	 torch.Size([1, 120])
BatchNorm1d output shape:	 torch.Size([1, 120])
Sigmoid output shape:	 torch.Size([1, 120])
Linear output shape:	 torch.Size([1, 84])
BatchNorm1d output shape:	 torch.Size([1, 84])
Sigmoid output shape:	 torch.Size([1, 84])
Linear output shape:	 torch.Size([1, 10])


### 2.3.2 比赛
* [COVID-19 Cases Prediction](https://www.kaggle.com/c/ml2021spring-hw1)
* [TIMIT framewise phoneme classification](https://www.kaggle.com/c/ml2021spring-hw2)

下面代码仅仅展示一个*最基础*的Baseline 代码：
* [比赛1](./covid19_prediction.ipynb)
* [比赛2](./phoneme_classification.ipynb)

并且下面将展示：
* 本地Windows环境（比赛1）下  （*由于这个数据集较小才采用这种方式，否则建议使用kaggle或者colab来跑程序*）
* Colab环境（比赛2）下如何下载数据集

首先安装kaggle官方库

In [ ]:
!pip install kaggle

登录[Kaggle个人信息版块](https://www.kaggle.com/)，点击“Create New API Token”下载kaagle.json文件:
![](https://s2.loli.net/2022/01/18/cF2IbDE7LKvUnkd.png)

随后新建'.kaggle'文件夹，将下载的json文件放入，并且整个文件夹移到C盘User目录下即可，最终如下所示:
![](https://s2.loli.net/2022/01/18/BHCn35UZ8sWEXAf.png)

**最后导入，kaggle包即可**

*注意*：用下面命令下载数据前，请务必先*同意该场比赛的规则*:
![](https://s2.loli.net/2022/01/18/HsbuUOEme6BK3o4.png)